# Terrain attributes extraction

Author: Thiago Nascimento (thiago.nascimento@eawag.ch)

This notebook is part of the EStreams publication and was used to extract and aggregate the terrain characteristics from the MERIT dataset.

* Note that this code enables not only the replicability of the current database but also the extrapolation to new catchment areas. 
* Additionally, the user should download and insert the original raw-data in the folder of the same name prior to run this code. 
* The original third-party data used were not made available in this repository due to redistribution and storage-space reasons.  

## Requirements
**Python:**

* Python>=3.6
* Jupyter
* geopandas=0.10.2
* numpy
* os
* pandas
* tqdm

Check the Github repository for an environment.yml (for conda environments) or requirements.txt (pip) file.

**Files:**

* data/terrain/riv_pfaf_2_MERIT_Hydro_v07_Basins_v01.shp. Available at: https://www.reachhydro.org/home/params/merit-basins (Last access: 23 November 2023)
* data/shapefiles/estreams_catchments.shp
* data/gee/terrain/EStreams_elevation_attributes_gee.csv. Elevation attributes CSV-file(s) exported from GEE.
* data/gee/EStreams_slope_attributes_gee.csv. Slope attributes CSV-file(s) exported from GEE.

**Directory:**

* Clone the GitHub directory locally
* Place any third-data variables in their respective directory.
* ONLY update the "PATH" variable in the section "Configurations", with their relative path to the EStreams directory. 

## References

* Yamazaki, D. et al. A high-accuracy map of global terrain elevations. Geophys Res Lett 44, 5844–5853 (2017).
* Yamazaki, D. et al. MERIT Hydro: A High-Resolution Global Hydrography Map Based on Latest Topography Dataset. Water Resour Res 55, 5053–5073 (2019).

## License
* MERIT: Dual-license - CC-BY-NC 4.0 & ODbL 1.0. http://hydro.iis.u-tokyo.ac.jp/~yamadai/MERIT_DEM/index.html (Last access: 27 November 2023)

## Observations
* This notebook assumes that the GEE code to export elevation and slope descriptors from the MERIT-dem dataset (EStreams_landscape_attributes_terrain_gee.txt) was run before in the GEE platform and that the output CSV-files are locally available. 

# Import modules

In [4]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import tqdm as tqdm
from utils.terrain import *

# Configurations

In [5]:
# Only editable variables:
# Relative path to your local directory
PATH = "../../.."

* #### The users should NOT change anything in the code below here.

In [6]:
# Non-editable variables:
PATH_OUTPUT = "results/staticattributes/"

# Set the directory:
os.chdir(PATH)

# Import data
## Catchment boundaries

In [7]:
catchment_boundaries = gpd.read_file('data/shapefiles/Catchment_Boundaries_HUGR_33new.shp')
catchment_boundaries.head()

,id,area_km2,outlet_lat,outlet_lng,name,area_offic,layer,path,area_diff,area_calc,basin_id,geometry
0,HUGR020,"9,600",46.785,21.142,6444410,9011,HUGR020,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,6.536,9595.794,HUGR020,"POLYGON ((21.13208 46.77291, 21.13208 46.77375..."
1,HUGR021,"189,000",46.423,18.896,6442080,189538,HUGR021,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,-0.284,188597.110,HUGR021,"POLYGON ((18.91708 46.41791, 18.91708 46.41625..."
2,HUGR022,"28,500",48.126,22.340,6444304,29057,HUGR022,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,-1.917,28507.473,HUGR022,"POLYGON ((22.32875 48.10875, 22.32791 48.10875..."
3,HUGR023,"188,000",46.627,18.869,6442060,189092,HUGR023,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,-0.577,188286.167,HUGR023,"POLYGON ((18.89041 46.62875, 18.88875 46.62708..."
4,HUGR025,"1,210",47.662,19.683,6444240,1222,HUGR025,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,-0.982,1206.441,HUGR025,"POLYGON ((19.68124 47.66875, 19.68291 47.66875..."


In [8]:
print("The total number of catchments to be processed are:", len(catchment_boundaries))

The total number of catchments to be processed are: 33


## Merit-Hydro river network

In [9]:
river_net_EU_MERIT = gpd.read_file('data/terrain/riv_pfaf_2_MERIT_Hydro_v07_Basins_v01.shp')
river_net_EU_MERIT

,COMID,lengthkm,lengthdir,sinuosity,slope,uparea,order,strmDrop_t,slope_taud,NextDownID,maxup,up1,up2,up3,up4,geometry
0,21000001,9.079822,5.620153,1.615583,0.000220,382.604819,2,2.0,0.000220,21000054,2,21000002,21000011,0,0,"LINESTRING (5.88917 47.94917, 5.88833 47.94917..."
1,21000002,2.842653,1.826319,1.556494,0.001124,325.410126,2,3.2,0.001124,21000001,2,21000004,21000013,0,0,"LINESTRING (5.90667 47.99833, 5.90750 47.99833..."
2,21000003,20.918003,14.119143,1.481535,0.001484,415.960268,2,31.1,0.001484,21000055,2,21000006,21000009,0,0,"LINESTRING (6.04750 47.95083, 6.04833 47.95083..."
3,21000004,12.775680,6.218682,2.054403,0.000609,275.794567,2,7.8,0.000609,21000002,2,21000005,21000016,0,0,"LINESTRING (5.91667 48.01333, 5.91750 48.01333..."
4,21000005,3.535374,2.233011,1.583232,0.001892,171.981367,2,6.7,0.001892,21000004,2,21000014,21000015,0,0,"LINESTRING (5.99083 48.03917, 5.99167 48.03833..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393459,29123032,30.204422,21.486454,1.405743,0.005751,82.869595,1,173.4,0.005751,0,0,0,0,0,0,"LINESTRING (43.49000 12.80583, 43.49083 12.805..."
393460,29123033,10.813198,8.296287,1.303378,0.014205,65.406667,1,153.2,0.014205,29122956,0,0,0,0,0,"LINESTRING (44.46250 12.82500, 44.46167 12.825..."
393461,29123034,16.082080,9.903307,1.623910,0.011262,67.604187,1,180.7,0.011262,0,0,0,0,0,0,"LINESTRING (44.59833 12.81833, 44.59750 12.819..."
393462,29123035,1.327687,0.656550,2.022219,0.000000,36.423539,1,0.0,0.000000,0,0,0,0,0,0,"LINESTRING (45.07917 12.90917, 45.07833 12.908..."


## GEE outputs

In [10]:
# Elevation descriptors
terrain_atrributes_gee_elevation = pd.read_csv("data/gee/terrain/EStreams_elevation_attributes_gee.csv", index_col=1)
terrain_atrributes_gee_elevation.drop(["system:index", ".geo"], axis = 1, inplace = True)
terrain_atrributes_gee_elevation.columns = ["ele_mt_max", "ele_mt_mean", "ele_mt_min"]
terrain_atrributes_gee_elevation

,ele_mt_mean,ele_mt_min,ele_mt_max
basin_id,,,
HUGR019,2272.107422,436.610990,78.054459
HUGR024,1472.909668,456.662286,144.930649
HUGR034,3759.748535,801.983360,86.414917
HUGR041,3759.748535,873.811544,100.683525
HUGR051,3759.748535,943.893692,120.313545
HUGR030,3026.098877,848.510382,135.520294
HUGR021,4020.411377,602.414872,77.173683
HUGR023,4020.411377,603.204350,77.173683
HUGR042,1564.308594,316.046150,82.805695


In [11]:
# Slope descriptors
terrain_atrributes_gee_slope = pd.read_csv("data/gee/terrain/EStreams_slope_attributes_gee.csv", index_col=1)
terrain_atrributes_gee_slope.drop(["system:index", ".geo"], axis = 1, inplace = True)
terrain_atrributes_gee_slope.columns = ["flat_area_fra", "slp_dg_mean", "steep_area_fra"]
terrain_atrributes_gee_slope = terrain_atrributes_gee_slope[["slp_dg_mean", "flat_area_fra", "steep_area_fra"]]
terrain_atrributes_gee_slope

,slp_dg_mean,flat_area_fra,steep_area_fra
basin_id,,,
HUGR019,8.676155,0.334287,0.213950
HUGR024,10.342121,0.197151,0.274640
HUGR034,12.738840,0.303692,0.385476
HUGR041,13.845216,0.256316,0.425633
HUGR051,14.994011,0.203130,0.466554
HUGR030,13.684028,0.217429,0.426043
HUGR021,8.062433,0.422437,0.183350
HUGR023,8.073854,0.421592,0.183642
HUGR042,7.683972,0.440756,0.212719


In [12]:
terrain_atrributes_df = pd.concat([terrain_atrributes_gee_elevation, terrain_atrributes_gee_slope], axis=1)
terrain_atrributes_df

,ele_mt_mean,ele_mt_min,ele_mt_max,slp_dg_mean,flat_area_fra,steep_area_fra
basin_id,,,,,,
HUGR019,2272.107422,436.610990,78.054459,8.676155,0.334287,0.213950
HUGR024,1472.909668,456.662286,144.930649,10.342121,0.197151,0.274640
HUGR034,3759.748535,801.983360,86.414917,12.738840,0.303692,0.385476
HUGR041,3759.748535,873.811544,100.683525,13.845216,0.256316,0.425633
HUGR051,3759.748535,943.893692,120.313545,14.994011,0.203130,0.466554
HUGR030,3026.098877,848.510382,135.520294,13.684028,0.217429,0.426043
HUGR021,4020.411377,602.414872,77.173683,8.062433,0.422437,0.183350
HUGR023,4020.411377,603.204350,77.173683,8.073854,0.421592,0.183642
HUGR042,1564.308594,316.046150,82.805695,7.683972,0.440756,0.212719


# Reproject to projected coordinates system

In [13]:
# Here you can check the crs of the datasets:
print("CRS of catchment_boundaries:", catchment_boundaries.crs)
print("CRS of river_net_EU_MERIT:", river_net_EU_MERIT.crs)

CRS of catchment_boundaries: epsg:4326
CRS of river_net_EU_MERIT: epsg:4326


In [14]:
# Define the target CRS to ETRS89 LAEA
target_crs = 'EPSG:3035'  # ETRS89 LAEA

# Reproject the GeoDataFrame to the target CRS
catchment_boundaries_reprojected = catchment_boundaries.to_crs(target_crs)
river_net_EU_MERIT_reprojected = river_net_EU_MERIT.to_crs(target_crs)

In [15]:
# Here you can check the new crs of the datasets:
print("CRS of catchment_boundaries:", catchment_boundaries_reprojected.crs)
print("CRS of river_net_EU_MERIT:", river_net_EU_MERIT_reprojected.crs)

CRS of catchment_boundaries: EPSG:3035
CRS of river_net_EU_MERIT: EPSG:3035


# Compute area in sqm

In [16]:
catchment_boundaries_reprojected["area_sqm"] = catchment_boundaries_reprojected.area
catchment_boundaries_reprojected.head()

,id,area_km2,outlet_lat,outlet_lng,name,area_offic,layer,path,area_diff,area_calc,basin_id,geometry,area_sqm
0,HUGR020,"9,600",46.785,21.142,6444410,9011,HUGR020,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,6.536,9595.794,HUGR020,"POLYGON ((5168393.183 2692574.791, 5168379.710...",9.595794e+09
1,HUGR021,"189,000",46.423,18.896,6442080,189538,HUGR021,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,-0.284,188597.110,HUGR021,"POLYGON ((5005386.762 2630548.758, 5005408.019...",1.885971e+11
2,HUGR022,"28,500",48.126,22.340,6444304,29057,HUGR022,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,-1.917,28507.473,HUGR022,"POLYGON ((5234673.724 2853771.530, 5234612.139...",2.850747e+10
3,HUGR023,"188,000",46.627,18.869,6442060,189092,HUGR023,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,-0.577,188286.167,HUGR023,"POLYGON ((5000655.811 2653584.729, 5000550.972...",1.882862e+11
4,HUGR025,"1,210",47.662,19.683,6444240,1222,HUGR025,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,-0.982,1206.441,HUGR025,"POLYGON ((5046159.839 2775900.149, 5046284.107...",1.206441e+09


# Disssolve niver-network


In [17]:
river_net_EU_MERIT_dissolved = river_net_EU_MERIT_reprojected.dissolve()

# River network density

In [18]:
# Create a spatial index for the river network
sindex = river_net_EU_MERIT_reprojected.sindex

# Initialize a dictionary to store results
results = {}

# Iterate through each catchment
for catchment_id in tqdm.tqdm(catchment_boundaries_reprojected.basin_id):

    # Filter the selected catchment
    selected_boundary = catchment_boundaries_reprojected[catchment_boundaries_reprojected['id'] == catchment_id]

    # Calculate the total length of lines within the selected catchment
    total_length = 0
    boundary_bounds = selected_boundary.total_bounds
    possible_matches_index = list(sindex.intersection(boundary_bounds))
    possible_matches = river_net_EU_MERIT_reprojected.iloc[possible_matches_index]
    
    for index, row in possible_matches.iterrows():
        if row['geometry'].intersects(selected_boundary.unary_union):
            total_length += row['geometry'].intersection(selected_boundary.unary_union).length

    # Store the result in the dictionary
    results[catchment_id] = total_length

# Convert the dictionary to a DataFrame
strm_dens_df = pd.DataFrame(list(results.items()), columns=['basin_id', 'totalnet_length_m'])
strm_dens_df.set_index("basin_id", inplace = True)
strm_dens_df

100%|██████████| 33/33 [00:37<00:00,  1.13s/it]


,totalnet_length_m
basin_id,
HUGR020,2.045944e+06
HUGR021,3.582148e+07
HUGR022,5.298104e+06
HUGR023,3.575938e+07
HUGR025,2.182385e+05
HUGR026,2.891406e+04
HUGR027,8.502362e+05
HUGR028,9.446325e+05
HUGR029,3.511096e+07


In [19]:
# Convert the dictionary to a DataFrame
strm_dens_df = pd.DataFrame(list(results.items()), columns=['basin_id', 'totalnet_length_m'])
strm_dens_df.set_index("basin_id", inplace = True)
strm_dens_df

,totalnet_length_m
basin_id,
HUGR020,2.045944e+06
HUGR021,3.582148e+07
HUGR022,5.298104e+06
HUGR023,3.575938e+07
HUGR025,2.182385e+05
HUGR026,2.891406e+04
HUGR027,8.502362e+05
HUGR028,9.446325e+05
HUGR029,3.511096e+07


In [20]:
strm_dens_df["area"] = catchment_boundaries_reprojected.set_index("basin_id").area
strm_dens_df["strm_dens"] = strm_dens_df["totalnet_length_m"] / strm_dens_df["area"] 
strm_dens_df

,totalnet_length_m,area,strm_dens
basin_id,,,
HUGR020,2.045944e+06,9.595794e+09,0.000213
HUGR021,3.582148e+07,1.885971e+11,0.000190
HUGR022,5.298104e+06,2.850747e+10,0.000186
HUGR023,3.575938e+07,1.882862e+11,0.000190
HUGR025,2.182385e+05,1.206441e+09,0.000181
HUGR026,2.891406e+04,1.096385e+08,0.000264
HUGR027,8.502362e+05,4.494402e+09,0.000189
HUGR028,9.446325e+05,5.773506e+09,0.000164
HUGR029,3.511096e+07,1.848107e+11,0.000190


# Enlongation ratio

In [21]:
# Create a dataframe to process the computation:
terrain_atrributes_enlon_ratio = pd.DataFrame()
terrain_atrributes_enlon_ratio["basin_id"] = catchment_boundaries_reprojected.basin_id
terrain_atrributes_enlon_ratio["area"] = catchment_boundaries_reprojected.area

# Assuming gdf is your GeoDataFrame with a Polygon geometry column named 'geometry'
terrain_atrributes_enlon_ratio['x_dimns'], terrain_atrributes_enlon_ratio['y_dimns'], terrain_atrributes_enlon_ratio['length']  = calculate_dimensions(catchment_boundaries_reprojected['geometry'])
terrain_atrributes_enlon_ratio

/Users/thiagomedeirosdonascimento/anaconda3/envs/estreams/lib/python3.9/site-packages/geopandas/geoseries.py:624: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)
/Users/thiagomedeirosdonascimento/anaconda3/envs/estreams/lib/python3.9/site-packages/geopandas/geoseries.py:624: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)
/Users/thiagomedeirosdonascimento/anaconda3/envs/estreams/lib/python3.9/site-packages/geopandas/geoseries.py:624: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead

,basin_id,area,x_dimns,y_dimns,length
0,HUGR020,9.595794e+09,178366.628778,160901.145253,178366.628778
1,HUGR021,1.885971e+11,917130.405045,479624.657558,917130.405045
2,HUGR022,2.850747e+10,297633.589066,305372.342001,305372.342001
3,HUGR023,1.882862e+11,921386.962708,483076.784911,921386.962708
4,HUGR025,1.206441e+09,53904.403212,66869.725144,66869.725144
5,HUGR026,1.096385e+08,18005.985729,20302.984617,20302.984617
6,HUGR027,4.494402e+09,135110.531176,115153.779382,135110.531176
7,HUGR028,5.773506e+09,159584.284107,159407.944101,159584.284107
8,HUGR029,1.848107e+11,945201.160680,518825.829729,945201.160680
9,HUGR030,1.304466e+10,302619.716913,190249.132770,302619.716913


In [22]:
# Enlongation ratio computation:
terrain_atrributes_enlon_ratio['elon_ratio'] = terrain_atrributes_enlon_ratio.apply(calculate_elongation_ratio, axis=1)
terrain_atrributes_enlon_ratio.set_index("basin_id", inplace = True)
terrain_atrributes_enlon_ratio

,area,x_dimns,y_dimns,length,elon_ratio
basin_id,,,,,
HUGR020,9.595794e+09,178366.628778,160901.145253,178366.628778,0.619701
HUGR021,1.885971e+11,917130.405045,479624.657558,917130.405045,0.534308
HUGR022,2.850747e+10,297633.589066,305372.342001,305372.342001,0.623885
HUGR023,1.882862e+11,921386.962708,483076.784911,921386.962708,0.531401
HUGR025,1.206441e+09,53904.403212,66869.725144,66869.725144,0.586109
HUGR026,1.096385e+08,18005.985729,20302.984617,20302.984617,0.581938
HUGR027,4.494402e+09,135110.531176,115153.779382,135110.531176,0.559889
HUGR028,5.773506e+09,159584.284107,159407.944101,159584.284107,0.537260
HUGR029,1.848107e+11,945201.160680,518825.829729,945201.160680,0.513209


# Final aggregation

In [23]:
# First we create an empty table data frame to assing the values to it
terrain_df = pd.DataFrame(index = catchment_boundaries_reprojected.basin_id)

# Now we proceed with the concatenation:
terrain_df = pd.concat([terrain_df, terrain_atrributes_df, terrain_atrributes_enlon_ratio.elon_ratio, 
                        strm_dens_df.strm_dens], axis=1)

terrain_df

,ele_mt_mean,ele_mt_min,ele_mt_max,slp_dg_mean,flat_area_fra,steep_area_fra,elon_ratio,strm_dens
basin_id,,,,,,,,
HUGR020,1832.485352,287.868877,78.024689,6.298653,0.524542,0.159708,0.619701,0.000213
HUGR021,4020.411377,602.414872,77.173683,8.062433,0.422437,0.183350,0.534308,0.000190
HUGR022,2272.107422,575.166733,99.266266,11.195253,0.203349,0.301992,0.623885,0.000186
HUGR023,4020.411377,603.204350,77.173683,8.073854,0.421592,0.183642,0.531401,0.000190
HUGR025,957.188232,265.355912,114.708008,6.112513,0.324909,0.058542,0.586109,0.000181
HUGR026,86.756012,84.764193,82.313629,0.168299,1.000000,0.000000,0.581938,0.000264
HUGR027,1941.960327,566.381848,150.036301,10.435450,0.172374,0.265764,0.559889,0.000189
HUGR028,602.170044,172.673123,98.146080,2.786013,0.683776,0.008093,0.537260,0.000164
HUGR029,4020.411377,611.621713,92.962807,8.188928,0.414202,0.186943,0.513209,0.000190


In [24]:
# Here we sort the columns:
terrain_df = terrain_df.sort_index(axis=0)
terrain_df

,ele_mt_mean,ele_mt_min,ele_mt_max,slp_dg_mean,flat_area_fra,steep_area_fra,elon_ratio,strm_dens
basin_id,,,,,,,,
HUGR019,2272.107422,436.610990,78.054459,8.676155,0.334287,0.213950,0.587927,0.000193
HUGR020,1832.485352,287.868877,78.024689,6.298653,0.524542,0.159708,0.619701,0.000213
HUGR021,4020.411377,602.414872,77.173683,8.062433,0.422437,0.183350,0.534308,0.000190
HUGR022,2272.107422,575.166733,99.266266,11.195253,0.203349,0.301992,0.623885,0.000186
HUGR023,4020.411377,603.204350,77.173683,8.073854,0.421592,0.183642,0.531401,0.000190
HUGR024,1472.909668,456.662286,144.930649,10.342121,0.197151,0.274640,0.600000,0.000175
HUGR025,957.188232,265.355912,114.708008,6.112513,0.324909,0.058542,0.586109,0.000181
HUGR026,86.756012,84.764193,82.313629,0.168299,1.000000,0.000000,0.581938,0.000264
HUGR027,1941.960327,566.381848,150.036301,10.435450,0.172374,0.265764,0.559889,0.000189


In [25]:
# Set the strm_dens to 1000km/km2 (improve units storage)
terrain_df.strm_dens = terrain_df.strm_dens*1000

In [26]:
# Assign the "basin_id" to the gauges names:
terrain_df.index.name = "basin_id"

In [ ]:
# Convert from frac to perc:
terrain_df.flat_area_fra = terrain_df.flat_area_fra * 100
terrain_df.steep_area_fra = terrain_df.steep_area_fra * 100
terrain_df

In [27]:
# Round the data to 3 decimals
terrain_df = terrain_df.astype(float).round(3)
terrain_df

,ele_mt_mean,ele_mt_min,ele_mt_max,slp_dg_mean,flat_area_fra,steep_area_fra,elon_ratio,strm_dens
basin_id,,,,,,,,
HUGR019,2272.107,436.611,78.054,8.676,0.334,0.214,0.588,0.193
HUGR020,1832.485,287.869,78.025,6.299,0.525,0.160,0.620,0.213
HUGR021,4020.411,602.415,77.174,8.062,0.422,0.183,0.534,0.190
HUGR022,2272.107,575.167,99.266,11.195,0.203,0.302,0.624,0.186
HUGR023,4020.411,603.204,77.174,8.074,0.422,0.184,0.531,0.190
HUGR024,1472.910,456.662,144.931,10.342,0.197,0.275,0.600,0.175
HUGR025,957.188,265.356,114.708,6.113,0.325,0.059,0.586,0.181
HUGR026,86.756,84.764,82.314,0.168,1.000,0.000,0.582,0.264
HUGR027,1941.960,566.382,150.036,10.435,0.172,0.266,0.560,0.189


# Data export

In [22]:
# Export the final dataset:
terrain_df.to_csv(PATH_OUTPUT+"estreams_terrain_attributes.csv")

# End